In [1]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel
from tqdm import tqdm

In [24]:
list(df_training_data["chr"].unique())

['chr1',
 'chr2',
 'chr3',
 'chr4',
 'chr5',
 'chr6',
 'chr7',
 'chr8',
 'chr9',
 'chr10',
 'chr11',
 'chr12',
 'chr13',
 'chr14',
 'chr15',
 'chr16',
 'chr17',
 'chr18',
 'chr19',
 'chr20',
 'chr21',
 'chr22',
 'chrX',
 'chrY',
 'chr11_KI270721v1_random',
 'chr14_KI270725v1_random',
 'chr14_KI270726v1_random',
 'chr15_KI270727v1_random',
 'chr16_KI270728v1_random',
 'chr1_KI270711v1_random',
 'chr22_KI270731v1_random',
 'chr22_KI270733v1_random',
 'chr22_KI270734v1_random',
 'chr1_KI270762v1_alt',
 'chr1_KI270766v1_alt',
 'chr1_GL383518v1_alt',
 'chr1_GL383519v1_alt',
 'chr1_GL383520v2_alt',
 'chr1_KI270763v1_alt',
 'chr1_KI270759v1_alt',
 'chr1_KI270761v1_alt',
 'chr2_KI270774v1_alt',
 'chr2_KI270769v1_alt',
 'chr2_GL582966v2_alt',
 'chr2_GL383522v1_alt',
 'chr2_KI270776v1_alt',
 'chr2_KI270767v1_alt',
 'chr3_JH636055v2_alt',
 'chr3_KI270777v1_alt',
 'chr3_KI270779v1_alt',
 'chr3_KI270782v1_alt',
 'chr4_GL000257v2_alt',
 'chr4_GL383527v1_alt',
 'chr5_KI270792v1_alt',
 'chr5_KI270791v

In [2]:
df_gencode = pd.read_csv("data/1-rmv_dup_introns_gencode_v44.tsv", sep="\t")
df_gencode = df_gencode.set_index(['strand', 'start', 'end'])
df_gencode = df_gencode.sort_index()

chrom_uniques = list(df_gencode["chr"].unique())  # we get the list of unique chromosome names
dict_chr = {}
# Splitting the dataframe chromosome groups separately into key(chromosome)-value(rows) in a dictionary
for chrom in tqdm(chrom_uniques):
	dict_chr[chrom] = df_gencode[df_gencode.chr == chrom]

dict_chr

100%|██████████| 24/24 [00:00<00:00, 76.15it/s]


{'chr18':                             chr
 strand start    end            
 +      11595    13151     chr18
                 15616     chr18
        13354    15616     chr18
        45235    45282     chr18
        45556    45640     chr18
 ...                         ...
 -      80160606 80202709  chr18
        80183150 80201963  chr18
                 80202709  chr18
        80202018 80202709  chr18
        80202932 80247276  chr18
 
 [7267 rows x 1 columns],
 'chr3':                              chr
 strand start     end            
 +      11799     14770      chr3
                  20556      chr3
                  23760      chr3
        12659     14770      chr3
                  20890      chr3
 ...                          ...
 -      198121656 198122027  chr3
        198122079 198122536  chr3
        198122645 198122729  chr3
        198122984 198123051  chr3
        198224435 198224520  chr3
 
 [24732 rows x 1 columns],
 'chr16':                             chr
 strand start

In [17]:
# Sample query: Find the nearest entry of chr1 with strand '+' and nearest to 13579
# query_chr = 'chr1'
# query_strand = '-'
# query_position = 14499

def find_nearest_match_start_ss(row, site):
    # print(row)
    query_chr = row["chr"]
    query_strand = row["strand"]
    query_position = row[site]

    # Filter the DataFrame for the relevant entries
    filtered_df = dict_chr[query_chr].loc[(query_strand)].copy()


    # Calculate the distance to the specified position
    filtered_df['distance'] = np.abs(filtered_df.index.get_level_values(site) - query_position)

    # Exclude exact match from the DataFrame (if it exists)
    filtered_df = filtered_df[filtered_df['distance'] != 0]

    # Find the row with the minimum distance (i.e., the nearest entry after excluding exact matches)
    nearest_entry = filtered_df.loc[filtered_df['distance'].idxmin()]

    return nearest_entry.distance

In [4]:
df_training_data = pd.read_csv("data/4.2_hg38_paired_introns.tsv", sep="\t", usecols=["chr","start","end","strand"])
df_training_data

,chr,start,end,strand
0,chr1,12227,12612,+
1,chr1,12721,13220,+
2,chr1,12057,12178,+
3,chr1,12697,12974,+
4,chr1,13052,13220,+
...,...,...,...,...
527913,chrY,24883840,24886132,+
527914,chrY,24888605,24889352,+
527915,chrY,24889386,24901111,+
527916,chrY,24833970,24840730,+


In [5]:
pandarallel.initialize()

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [20]:
%%time
df_training_data["nearest_start_ss_dist"] = df_training_data.parallel_apply(find_nearest_match_start_ss, args=("start",), axis=1)

KeyError: 'chr6_GL000255v2_alt'

In [19]:
# %%time
#
# df_test = df_training_data.copy().head(10000)
# df_test["start_site_nearest"] = df_test.parallel_apply(find_nearest_match_start_ss, args=("start",), axis=1)
# df_test

CPU times: user 29.4 ms, sys: 18.2 ms, total: 47.6 ms
Wall time: 1.37 s


,chr,start,end,strand,start_site_nearest
0,chr1,12227,12612,+,170
1,chr1,12721,13220,+,24
2,chr1,12057,12178,+,170
3,chr1,12697,12974,+,24
4,chr1,13052,13220,+,322
...,...,...,...,...,...
9995,chr1,41848445,41848870,-,644
9996,chr1,41849089,41918412,-,644
9997,chr1,41918524,42035806,-,52909
9998,chr1,41849089,41864577,-,644


In [21]:
df_training_data

,chr,start,end,strand
0,chr1,12227,12612,+
1,chr1,12721,13220,+
2,chr1,12057,12178,+
3,chr1,12697,12974,+
4,chr1,13052,13220,+
...,...,...,...,...
527913,chrY,24883840,24886132,+
527914,chrY,24888605,24889352,+
527915,chrY,24889386,24901111,+
527916,chrY,24833970,24840730,+
